In [1]:
import ee
import geemap
import geopandas as gpd
import numpy as np
from agroforestry.config import * 
from agroforestry.geeHelpers import *
from agroforestry.naipProcessing import *
from agroforestry.snicProcessing import *
from agroforestry.randomForest import *
from agroforestry.exportFunctions import *

In [ ]:
#print

In [2]:
# establish connection with ee account. might require some additional configuration based on local machine 
try:
        ee.Initialize()
except Exception as e:
        ee.Authenticate()
        ee.Initialize()# i

In [5]:
# define the aoi
aoiID = initGridID # something to itorate over for now is defined based on the input training dataset 
# or manually define it for where you want to apply the model too
# aoiID = "X12-601"
# # this becomes the AOI to used in the prepNAIP function. I'll need to edit it so that it converts the input data into a bbox 
gridSelect = grid.loc[grid.Unique_ID == aoiID]


#Define bands to use
bandsToUse = vsurfWithCor

# select multiple grids level 1 
## want to pull thing from the csv rather than write it out. 
# aoiID = ["X12-555","X12-556","X12-557","X12-600","X12-601","X12-602","X12-645","X12-646","X12-647"]
# gridSelect =  grid.loc[grid.Unique_ID.isin(aoiID)].dissolve()
# gridSelect = grid.loc[grid.Unique_ID in aoiID]
# dissolve into a single object
gridSelect





,Unique_ID,geometry
149,X12-150,"MULTIPOLYGON (((-96.57547 40.52184, -96.40156 ..."


In [6]:
# convert to a gee object 
aoi1 = geemap.gdf_to_ee(gridSelect)
# create a sub grid for downloading 
# downloadGrids = geemap.fishnet(aoi1.geometry(), rows=6, cols=4, delta=0)
aoiID

'X12-150'

In [7]:
Map = geemap.Map(center=(42.3, -98), zoom=10)
Map.add_basemap('HYBRID')
Map.addLayer(aoi1, {'color': '000000ff',
                    'width': 2,
                    'lineType': 'solid'},
             'area of interest')
# Map.addLayer(downloadGrids, {'color': '000000ff',
#                     'width': 2,
#                     'lineType': 'solid'},
#              'area of subgrid')

Map

Map(center=[42.3, -98], controls=(WidgetControl(options=['position', 'transparent_bg'], widget=SearchDataGUI(c…

In [9]:
# import training dataset 
# trainingData = gpd.read_file(filename="data/processed/trainingdataset_withClasses.geojson")
trainingData = gpd.read_file(filename="data/processed/" + str(initGridID) +"/"+ "agroforestrySamplingData_"+str(year)+".geojson") # initGridID defined int he config file

# print(type(trainingData))
# select the training class of interest and drop unnecessary columns
trainingSubset = trainingData
# trainingSubset =  trainingData[trainingData.sampleStrat == "original"] ## will want to drop this as we wont have multiple sampling categories 
# print(trainingSubset)
# convert to ee object
pointsEE = geemap.gdf_to_ee(gdf=trainingSubset)
# subset testing and training data 
training = pointsEE.filter(ee.Filter.gt('random', test_train_ratio))
testing = pointsEE.filter(ee.Filter.lte('random',test_train_ratio))
# traing the rf model 
# rfCluster = trainRFModel(bands=bandsToUse_Cluster, inputFeature=training, nTrees=nTrees,setSeed=setSeed)
# rfPixel = trainRFModel(bands=bandsToUse_Pixel, inputFeature=training, nTrees=nTrees,setSeed=setSeed)
rfPixelTrim = trainRFModel(bands=bandsToUse,  inputFeature=training, nTrees=nTrees,setSeed=setSeed )
## run validation using the testing set 
# clusterValidation = testRFClassifier(classifier=rfCluster, testingData= testing)
# pixelValidation = testRFClassifier(classifier=rfPixel, testingData= testing)
pixelValidationTrim = testRFClassifier(classifier=rfPixelTrim, testingData= testing)


In [14]:
trainingData



,B,B_mean,G,G_mean,N,N_mean,R,R_mean,contrast_g,contrast_g_mean,...,entropy_n,entropy_n_mean,gridID,id,nd,nd_mean,nd_mean_neighborhood,nd_sd_neighborhood,presence,geometry
0,108,73.782608,123,100.819130,114,135.664352,135,88.839996,416.195657,478.451447,...,6.068239,6.070603,X12-150,1_0,-0.084337,0.213948,0.220796,0.137951,1,POINT (-96.51443 40.59645)
1,64,74.534714,110,112.472595,157,156.282578,82,93.241173,274.271944,284.416382,...,6.046516,5.904170,X12-150,1_1,0.313808,0.263079,0.314559,0.071999,1,POINT (-96.51497 40.59644)
2,73,62.985073,103,85.039803,142,126.568825,84,74.271973,391.643727,405.787048,...,6.134047,6.059346,X12-150,1_2,0.256637,0.266873,0.252676,0.084276,1,POINT (-96.51513 40.59620)
3,64,62.304527,92,87.481483,142,135.897125,86,71.847740,325.624368,340.271912,...,6.010713,6.054325,X12-150,1_3,0.245614,0.314021,0.265334,0.091434,1,POINT (-96.51510 40.59593)
4,78,78.454880,121,112.807785,158,148.159454,91,95.635971,385.298139,527.601013,...,6.017210,6.019384,X12-150,1_4,0.269076,0.228766,0.284527,0.086953,1,POINT (-96.51350 40.59637)
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
700,102,85.870773,124,121.373512,123,132.281281,111,108.804108,55.806181,105.009926,...,5.406288,5.498817,X12-150,2_398,0.051282,0.099320,0.055730,0.055527,,POINT (-96.40741 40.53161)
701,72,73.632187,102,105.217346,173,175.609192,75,74.261230,127.633100,111.793823,...,5.280856,5.456089,X12-150,2_399,0.395161,0.407125,0.415836,0.043821,,POINT (-96.40775 40.53446)
702,71,72.427658,109,103.865334,186,178.201996,75,72.159233,90.550839,97.602905,...,5.234920,5.433488,X12-150,2_400,0.425287,0.424028,0.431352,0.032518,,POINT (-96.40741 40.53451)
703,73,73.504265,104,106.988640,184,178.598007,73,74.308952,81.533146,97.752411,...,4.989983,5.322830,X12-150,2_401,0.431907,0.413388,0.435851,0.029706,,POINT (-96.40826 40.53446)


In [ ]:
# cant print tuple with this function
geePrint(pixelValidationTrim)

In [ ]:

# define export aoi
# exportAOI = ee.Feature(downloadGrids.toList(50).get(4))
# geePrint(exportAOI)
# exportAOI.geometry()
year


In [12]:
### 
# Generate model based on year define in config 

# generate NAIP layer 
naipEE = prepNAIP(aoi=aoi1, year=year,windowSize=windowSize)

# normal the naip data
# normalizedNAIP = normalize_by_maxes(img=naipEE, bandMaxes=bandMaxes)

# produce the SNIC object 
## filtering the image bands right away based on the single model output 
snicData = snicOutputs(naip = naipEE,
                       SNIC_SeedShape = SNIC_SeedShape, 
                       SNIC_SuperPixelSize = SNIC_SuperPixelSize, 
                       SNIC_Compactness = SNIC_Compactness, 
                       SNIC_Connectivity = SNIC_Connectivity,
                       # nativeScaleOfImage = nativeScaleOfImage, 
                       bandsToUse_Cluster = bandsToUse_Cluster).select(bandsToUse)
# apply the model and clip to aoi and reclass to unsigned 8bit image 
classifiedPixelsTrim = applyRFModel(imagery=snicData, bands=bandsToUse,classifier=rfPixelTrim).clip(aoi1).uint8()
# produce image for map 
demoImage = classifiedPixelsTrim #.clip(exportAOI)
# demoImage = classifiedPixelsTrim.clip(ee.Feature(downloadGrids.toList(50).get(10))).reproject(crs='EPSG:4326', scale=5)

geePrint(demoImage)


EEException: No valid training data were found.

In [ ]:
# print
# vsurfNoCor
# vsurfWithCor

In [ ]:
# Set visualization parameters.
vis_params = {
    'min': 0,
    'max': 1,
    'palette': ['#f5f7f710', '#10c9a1'],
}
# add features to the map
Map.addLayer(demoImage, vis_params, str(year) + ' model')
# Map.addLayer(demoImage2010, vis_params,'2010 model',False)
# Map.addLayer(demoImage2020, vis_params,'2020 model',False)

Map

In [ ]:
# export image to asset 
task = ee.batch.Export.image.toAsset(
  image = demoImage,
  description = 'fullAOI_level1surround2020',
  assetId = "projects/agroforestry2023/assets/testExport_10grids2020",
  region=aoi1.geometry(),
  scale=1,
  crs= demoImage.projection(),
  maxPixels = 1e13
)
task.start()

In [ ]:
# track the task 
import time
while task.active():
  print('Polling for task (id: {}).'.format(task.id))
  time.sleep(5)

In [ ]:
geePrint(demoImage)